# 챔피언 시너지 또는 2:2구도 분석

- blue팀 정글포지션인 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONNAME -> j_champ) (테이블 이름 - blue_jungle)
- blue팀 탑포지션인 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONNAME -> t_champ) (테이블 이름 - blue_top)
- blue팀끼리 merge 

- red 동일하게 정글, 탑 뽑아내서 merge


- blue팀과 red팀의 데이터를 merge하기 위해서 enemy_blue / enemy_red 라는 데이터프레임 제작 
- 이후 championName들을 rename을 통해 각각 enemy_j_champ_cham enemy_t_champ으로 변환


- blue팀입장에서 red팀의 데이터 merge
- red 팀 입장에서 blue팀 데이터 merge

- 위에서 만들어진 merge테이블을 concat을 이용해서 하나의 데이터 테이블로 만들기
- games와 win의 횟수를 구하고 win_rate(승률) 컬럼을 추가한 result 테이블 

In [1]:
import pandas as pd
from my_utils import *

In [2]:
conn = connect_mysql()
query = 'select * from lol_datas'
df =pd.DataFrame(sql_execute_dict(conn,query))
conn.close()

In [6]:
blue_jungle = df[(df.teamId == 100) &(df.teamPosition == 'JUNGLE')][['gameId','championName','win']].\
              rename(columns = {'championName':'j_champ'})
blue_top = df[(df.teamId == 100) &(df.teamPosition == 'TOP')][['gameId','championName','win']].\
              rename(columns = {'championName':'t_champ'})

In [7]:
blue_team = pd.merge(blue_jungle,blue_top,on =['gameId','win'])

In [8]:
red_jungle = df[(df.teamId == 200) &(df.teamPosition == 'JUNGLE')][['gameId','championName','win']].\
              rename(columns = {'championName':'j_champ'})
red_top = df[(df.teamId == 200) &(df.teamPosition == 'TOP')][['gameId','championName','win']].\
              rename(columns = {'championName':'t_champ'})

In [9]:
red_team = pd.merge(red_jungle,red_top,on =['gameId','win'])

In [10]:
blue_tmp = blue_team[['gameId','j_champ','t_champ']].rename(columns = {'j_champ':'enemy_j_champ','t_champ':'enemy_t_champ'})


In [11]:
red_tmp = red_team[['gameId','j_champ','t_champ']].rename(columns = {'j_champ':'enemy_j_champ','t_champ':'enemy_t_champ'})

In [13]:
blue_result = pd.merge(blue_team,red_tmp,on=['gameId'])

In [14]:
red_result = pd.merge(red_team,blue_tmp,on=['gameId'])


In [15]:
tmp = pd.concat([blue_result,red_result])


In [16]:
tmp['win'] = tmp.apply(lambda x:1 if x.win =='True' else 0 ,axis=1)


In [17]:
tmp_df = tmp.drop('gameId',axis=1)


In [18]:
games = tmp_df.groupby(['j_champ','t_champ','enemy_j_champ','enemy_t_champ']).count().rename(columns = {'win':'cnt'})


In [19]:
wins = tmp_df.groupby(['j_champ','t_champ','enemy_j_champ','enemy_t_champ']).sum().rename(columns = {'win':'win_cnt'})


In [20]:
result = games.join(wins)


In [21]:
result['win_rate'] = round(result.win_cnt / result.cnt,2)


In [22]:
result['win_rate'] = round(result.win_cnt / result.cnt,2)


In [23]:
result.sort_values('cnt',ascending=False)


cnt  win_cnt  win_rate
j_champ  t_champ enemy_j_champ enemy_t_champ                        
LeeSin   Yone    Graves        Aatrox           5        1      0.20
Graves   Aatrox  LeeSin        Yone             5        4      0.80
LeeSin   Jax     Nocturne      Aatrox           4        1      0.25
Nocturne Aatrox  LeeSin        Jax              4        3      0.75
Taliyah  Aatrox  LeeSin        Rumble           3        2      0.67
...                                           ...      ...       ...
JarvanIV Trundle Nidalee       Illaoi           1        1      1.00
                 Nocturne      Rumble           1        1      1.00
                 Rammus        Kayn             1        1      1.00
                 RekSai        Gangplank        1        0      0.00
Zyra     Yone    LeeSin        Jayce            1        1      1.00

[12588 rows x 3 columns]

### 닉네임과 tag를 통해 매치데이터를 임의의 숫자만큼 가져와서 그사람이 플레이한 챔피언들의 데이터프레임 만들기
### 결과컬럼 - [챔피언이름 , 전체게임횟수, 승,패,승률,kills,deaths,assists,kda]

In [24]:
import requests
from tqdm import tqdm
import pandas as pd

In [25]:
name = '괴물쥐'
tag = 'KR3'
api_key = 'RGAPI-3e834f86-cece-4e8a-9399-3b8110fcbe60'
num = 50
queue = 420

In [27]:
url = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}?api_key={api_key}'

In [28]:
puuid = requests.get(url).json()['puuid']

In [29]:
url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue}&start=0&count={num}&api_key={api_key}'

In [30]:
df_create = []
for match_id in tqdm(requests.get(url2).json()):
    url3 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    matches = requests.get(url3).json()
    tmp = []
    try:
        for match in matches['info']['participants']:
            if match['riotIdGameName']== name and match['riotIdTagline'] == tag:
                tmp.append(match['championName'])
                tmp.append(match['win'])
                tmp.append(match['kills'])
                tmp.append(match['deaths'])
                tmp.append(match['assists'])
        df_create.append(tmp)
    except:
        try:
            for match in matches['info']['participants']:
                if match['riotIdName']== name and match['riotIdTagline'] == tag:
                    tmp.append(match['championName'])
                    tmp.append(match['win'])
                    tmp.append(match['kills'])
                    tmp.append(match['deaths'])
                    tmp.append(match['assists'])
            df_create.append(tmp)
        except:
            continue

100%|██████████| 50/50 [00:27<00:00,  1.83it/s]


In [39]:
df = pd.DataFrame(df_create,columns = ['championName','win','kills','deaths','assists'])

In [40]:
df

,championName,win,kills,deaths,assists
0,Ezreal,True,5,7,7
1,Ezreal,False,3,6,5
2,Varus,True,13,2,10
3,Kaisa,False,3,6,3
4,Xayah,False,4,4,0
5,Kalista,True,5,0,3
6,Lucian,True,11,0,4
7,TwistedFate,True,9,8,12
8,Nocturne,False,2,2,3
9,Ezreal,True,12,2,6


In [41]:
cnt_df = df[['championName','win']].groupby('championName').count().rename(columns = {'win':'cnt'})

In [42]:
sum_df = df.groupby('championName').sum()

In [43]:
result_df = pd.merge(sum_df,cnt_df,on='championName')

In [45]:
result_df['lose'] = result_df.cnt - result_df.win

In [46]:
result_df['win_rate'] = round(result_df.win / result_df.cnt,2)*100

In [47]:
result_df['kda'] = round((result_df.kills + result_df.assists)/result_df.deaths,2)

In [48]:
result_df[['cnt','win','lose','win_rate','kills','deaths','assists','kda']].sort_values(by=['cnt','win_rate'],ascending=False)

,cnt,win,lose,win_rate,kills,deaths,assists,kda
championName,,,,,,,,
Ezreal,15,9,6,60.0,115,62,116,3.73
Kaisa,7,2,5,29.0,42,33,33,2.27
Xayah,5,2,3,40.0,46,29,27,2.52
Caitlyn,4,1,3,25.0,21,21,29,2.38
Samira,3,3,0,100.0,42,21,22,3.05
Zeri,3,2,1,67.0,16,8,18,4.25
Kalista,3,1,2,33.0,19,18,18,2.06
Varus,3,1,2,33.0,19,7,27,6.57
Lucian,2,2,0,100.0,26,5,12,7.60


★ 미니프로젝트 과제 

- API를 통해서 라이엇 데이터 수집하기(로우데이터 DF만들기) - 티어는 자유
- 불러온 데이터를 전처리하기(중복제거, na값 제거) matches,timeline 합쳐서 최소컬럼 30개 
- 위의 과정을 함수화해서 모듈 만들기



- 전처리된 로우데이터를 mysql에 저정하기 -> 최소 5만개
- 팀과 함께 상의해서 흥미롭고 재밌을것 같은 주제 3가지 선정
- 지표 분석을 통해 나온 df를 시각화 모듈을 사용해서 표현하세요 (3개중 2개 필수)
- 전처리부터 시각화까지 캡쳐해서 폴더에 저장
- ppt를 만들어서 간단하게 발표 예정

- 모든 py파일, ipynb파일, 사진, 개발보고서, ppt 다 하나의 폴더로 압축
- appfa@gameeye.co.kr
- 목요일 점심 전까지 오후에 발표